In [ ]:
# Determined time invariant parameters from PEM model (Lui et al. 2022)
# The prediction error method (PEM) is used to estimate the time invariant parameters of the model 
# according to Yu et al. 2018
# Ei, Di, Fi, X, Vi and Vo are the parameters to be estimated

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy

from constant_file import *
from Monitored_file import *
from scipy.signal import cont2discrete
from scipy.optimize import minimize  

# System identification package for black-box model
# import sysidentpy as sysidpy
# from sysidentpy.basis_function import Polynomial
import control as cnt

# import relavant packages for sippy and check for file paths
# try:
#     from sippy_unipi import system_identification
# except ImportError:
#     import os
#     import sys

#     sys.path.append(os.pardir)
#     from sippy_unipi import system_identification

# from sippy_unipi import functionset as fset
# from sippy_unipi import functionsetSIM as fsetSIM

# assume the coeeficients are known and constant for all servers
E_arr = np.full((NUMBER_OF_SERVERS,1), Ei)  # initial guess for Ei
#D_arr = np.full((NUMBER_OF_SERVERS,1), Di)  # initial guess for Di
D_arr = Di.reshape((NUMBER_OF_SERVERS,1))  # initial guess for Di
F_arr = np.full((NUMBER_OF_SERVERS,1), Fi)  # initial guess for Fi

# Physics for the PEM model
# Version 2: Linear SS from control-oriented model paper by Wang et al. 2025
# for a short period, the data center model is found to be linear (ref: Jiagn et al 2021 (eq.1))
# Array for temperature parameter
TI = np.zeros(10)    # intial inlet temperature
TS = np.zeros(10)    # intial server temperature
TO = np.zeros(10)    # intial outlet temperature


TRCU = TRCU_0  # intial RCU outlet temperature
QRCU = QRCU_0  # intial RCU airflow
PS = np.zeros(10)    # intial server power consumption

# Array for coefficients parameters
QS = np.zeros(10)    # Air flow rate through virtually aggregrated server
QOC = np.zeros(10)  # Air flow rate through virtually aggregrated server
QOH = np.zeros(10)  # Air flow rate through virtually aggregrated server

# intitialisation for all servers at t = 0
TI[:] = TI_10
TO[:] = TO_10
TS[:] = TS_10
PS[:] = PS_0      # 10 servers in a rack with 500 watts peak each

# def evaluate_flows(inlet_temp, QRCU):
#     global QS, QL, QOC, QOH
#     i = 0 # i  here signify the server index from 1 to 10

#     # determine Qs,i
#     for i in range(10):
#         if inlet_temp[i] <= 27:
#             QS[i] = 0.05852
#         elif (inlet_temp[i] > 27 and inlet_temp[i] <= 35):
#             QS[i] = 0.05852 + (inlet_temp[i]-27)*0.00528
#         else:
#             QS[i] = 0.01005
#         i = i + 1

#     # Detmerine QL. QOC and QOH
#     QL = np.sum(QS) - QRCU

#     # Evaluate QOC from the first server
#     i = 0 
#     for i in range(10):
#         if i == 0:
#             QOC[i] = D_arr[i] * QRCU + E_arr[i] * np.abs(QL) - F_arr[i] * np.abs(QL) - QS[i]
#         elif i > 0 and i < 9:
#             QOC[i] = QOC[i-1] + D_arr[i] * QRCU + E_arr[i] * np.abs(QL) - F_arr[i] * np.abs(QL) - QS[i]
#         else:
#             QOC[i] = QOC[i-1] + D_arr[i] * QRCU + E_arr[i] * np.abs(QL) - F_arr[i] * np.abs(QL) - QS[i]
#         i = i + 1
        
#     # Evaluate QOH from the last server    
#     i = 9
#     for i in reversed(range(10)):
#         if i == 0:
#             QOH[i] = QOH[i+1] + F_arr[i] * np.abs(QL) + QS[i]
#         elif i > 0 and i < 9:
#             QOH[i] = QOH[i+1] + Fi * np.abs(QL) + QS[i]
#         else:
#             QOH[i] = Fi * np.abs(QL) + QS[i]
#         i = i - 1

# Obtain measured Qrcu and Trcu 
Qrcu = np.zeros(data_length)       # List of measured RCU airflow
Trcu = np.zeros(data_length)       # List of measured RCU outlet temperature
Trcu = RCU_TEMPERATURE   # measured RCU exhaust temperature from dataset

# Qrcu is a function of the server temperature according to the cooling contol logic Lui et al. 2022
t = 0
for t in range(len(Trcu)):
    if Trcu[t] < 35:
        Qrcu[t] = 0.11
    elif Trcu[t] >= 35 and Trcu[t] < 40:
        Qrcu[t] = 0.11 + 0.0022*(Trcu[t]-35)
    else:
        Qrcu[t] = float(np.nan)  # Beyond this point, the cooling system cannot maintain the desired temperature

# Assign each element to matrix
Ac = np.zeros((3*N,3*N))
Bc = np.zeros((3*N,N+2))
Cc = np.zeros((3*N,3*N))
Dc = np.zeros((3*N,N+2))
state_offset  = np.zeros((3*N,1)) #Offset from linearisation output
output_offset = np.zeros((3*N,1)) #Offset from linearisation

i = 0
# Assign linear offset from the linearisation of multiplicative term D*TRCU*QRCU/VI
# this linearisation is done every time before the matrices get update
for i in range(3*N):
    state_offset[i] = (D_arr[i%3] / INLET_VOLUME)*Trcu[i]*Qrcu[i]

# ---------- helpers ----------
def i_TI(i, N): return i
def i_TS(i, N): return N + i
def i_TO(i, N): return 2*N + i

def fan_qs_piecewise(TI):
    # per-server flow per kW @ inlet TI (piecewise)
    if TI <= 27.0:         return 0.05852
    elif TI < 35.0:        return 0.05852 + (TI - 27.0)*0.00528
    else:                  return 0.1005

def flows_at_op(TI_op, QRCU_op, D, E, F): # Flow at the operating temperature point
    N = len(TI_op)
    QS = np.array([fan_qs_piecewise(TI_op[i]) for i in range(N)])
    QL = QS.sum() - QRCU_op

    # QOC forward
    QOC = np.zeros(N)
    QOC[0] = D[0]*QRCU_op + E[0]*abs(QL) - F[0]*abs(QL) - QS[0]
    for i in range(1, N):
        QOC[i] = QOC[i-1] + D[i]*QRCU_op + E[i]*abs(QL) - F[i]*abs(QL) - QS[i]

    # QOH backward
    QOH = np.zeros(N)
    QOH[N-1] = F[N-1]*abs(QL) + QS[N-1]
    for i in range(N-2, -1, -1):
        QOH[i] = QOH[i+1] + F[i]*abs(QL) + QS[i]

    return QS, QL, QOC, QOH

def build_continuous_AB(D, E, F, TI_op, TRCU_op, QRCU_op, VI, VO, Xth):
    """
    theta = [D, E, F]  (scalars for simplicity; extend to per-server if needed)
    VI, VO: inlet/outlet control volumes
    Xth: thermal inertia (J/K) equivalent (server exhaust zone)
    """
    N = len(TI_op)
    QS, QL, QOC, QOH = flows_at_op(TI_op, QRCU_op, D, E, F)

    Ac = np.zeros((3*N, 3*N))
    Bc = np.zeros((3*N, N+2))  # [QRCU, TRCU, PS1..PSN]

    # --- Ac matrix ---
    for i in range(N):
        # dTI_i
        if i > 0:
            Ac[i_TI(i,N), i_TI(i-1,N)] = - QOC[i-1] / VI     # upstream TI
        Ac[i_TI(i,N), i_TI(i,N)]     = (- F[i]*abs(QL) - QOC[i]) / VI
        Ac[i_TI(i,N), i_TS(i,N)]     = - QS[i] / VI
        Ac[i_TI(i,N), i_TO(i,N)]     = + E[i]*abs(QL) / VI

        # dTS_i
        Ac[i_TS(i,N), i_TI(i,N)] =   QS[i] / Xth
        Ac[i_TS(i,N), i_TS(i,N)] = - QS[i] / Xth

        # dTO_i
        Ac[i_TO(i,N), i_TI(i,N)] =   F[i]*abs(QL) / VO
        Ac[i_TO(i,N), i_TS(i,N)] =   QS[i] / VO
        Ac[i_TO(i,N), i_TO(i,N)] = (- E[i]*abs(QL) - QOH[i]) / VO
        if i < N-1:
            Ac[i_TO(i,N), i_TO(i+1,N)] = QOH[i+1] / VO  # coupling to downstream TO

    # --- Bc matrix ---
    # TI_i rows: dependence on rack-level (QRCU, TRCU)
    for i in range(N):
        Bc[i_TI(i,N), 0] += (D[i]/VI) * TRCU_op   # QRCU column
        Bc[i_TI(i,N), 1] += (D[i]/VI) * QRCU_op   # TRCU column
    # TS_i rows: per-server PS_i inputs at columns 2+i
    for i in range(N):
        Bc[i_TS(i,N), 2+i] = 1.0 / Xth
    # TO rows: no direct input here in this linearization (leave zeros)
    return Ac, Bc, QS, QL, QOC, QOH

def discretize(Ac, Bc, dt):
    Cc = np.eye(Ac.shape[0])
    Dc = np.zeros((Ac.shape[0], Bc.shape[1]))
    Ad, Bd, Cd, Dd, _ = cont2discrete((Ac, Bc, Cc, Dc), dt, method='zoh')
    return Ad, Bd, Cd, Dd

# Define the input and state vectors
# X = np.zeros((3*N,1))  # State vector [TI; TS; TO]
# U = np.zeros((N+2,1))  # Input vector [Qrcu; Trcu; Ps]
# Y = np.zeros((3*N,1))  # Output vector [TI; TS; TO]
# Y_hat = np.zeros((3*N,1))  # Predicted output vector [TI; TS; TO]


# plot the A matrix single element evolution over time
A_previous = np.zeros((3*N,3*N)) 
plt.figure(figsize=(12, 8))
for i in range(len(Xmeas.T)):
    CURRENT_INTERVAL = i
    TI = INLET_TEMPERATURE[CURRENT_INTERVAL]   
    Ac, Bc, QS, QL, QOC, QOH = build_continuous_AB(D_arr, E_arr, F_arr, TI, TRCU, QRCU, INLET_VOLUME, OUTLET_VOLUME, THERMAL_INERTIA)
    A, B, C, D = discretize(Ac, Bc, SAMPLING_TIME)
    A_11 = A[1, 1]  # Example: tracking the (1,1) element of A matrix
    plt.scatter(i, A_11, color='blue')  # Scatter plot for each time step
    plt.scatter(i, A_previous[1, 1]-A_11, color='red') # Change in A matrix element
    A_previous = A
plt.title('A matrix element evolution over time')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.grid()
plt.show()

# Criteria to update the matrices usign sensitivity analysis


# Store matrix elements to excel file
df = pd.DataFrame(Ac)
df.to_excel("Ac_matrix.xlsx", index=False)

df = pd.DataFrame(A)
df.to_excel("A_matrix.xlsx", index=False)

df = pd.DataFrame(Bc)
df.to_excel("Bc_matrix.xlsx", index=False)

df = pd.DataFrame(B)
df.to_excel("B_matrix.xlsx", index=False)

# Plot datasets
# plot the example datasets
plt.figure(figsize=(12, 8))
plt.subplot(3, 1, 1)
plt.plot(TIME_STAMP, INLET_TEMPERATURE)
plt.title('Inlet Temperature')
plt.xlabel('Time')
plt.ylabel('Temperature (°C)')
plt.grid()

plt.subplot(3, 1, 2)
plt.plot(TIME_STAMP, SERVER_TEMPERATURE)
plt.title('Server Temperature')
plt.xlabel('Time')      
plt.ylabel('Temperature (°C)')
plt.grid()

plt.subplot(3, 1, 3)
plt.plot(TIME_STAMP, OUTLET_TEMPERATURE)
plt.title('Outlet Temperature')
plt.xlabel('Time')
plt.ylabel('Temperature (°C)')
plt.grid()
plt.tight_layout()
plt.show()



# implementing PEM using python packages SysIdentPy
# Step1: Estimate the model using state space and initial guess values

# Extract params values
initial_params = [Ei, Di, Fi, THERMAL_INERTIA, INLET_VOLUME, OUTLET_VOLUME]

# Define cost function for PEM
def PEM(params, U, Ac, Bc, N, maxiter):
    # Extract parameters from the initial guess values
    Ei = params.get("Ei", 0)
    Di = params.get("Di", 0)
    Fi = params.get("Fi", 0)
    thermal_inertia = params.get("X", 0)
    Vi = params.get("Vi", 0)
    Vo = params.get("Vo", 0)

    # Evaluate matrices A and B at the initial guess values
    Ac, Bc = build_continuous_AB(D_arr, E_arr, F_arr, TI, TRCU, QRCU, INLET_VOLUME, OUTLET_VOLUME, THERMAL_INERTIA)
    global Trcu, Qrcu     # Extract measured Qrcu from the calculated values above (from Trcu)

    # state vector X = [TI1, TS1, TO1, TI2, TS2, TO2, ..., TI10, TS10, TO10]
    # input vector U = [Qrcu, Trcu, PS1, PS2, ..., PS10]
    num_times =  INLET_TEMPERATURE.shape[0]
    num_servers = INLET_TEMPERATURE.shape[1]
    X = np.zeros((num_times, num_servers * 3))  # shape: (num_times, 30)
    U = np.zeros((num_times, num_servers + 2))  # shape: (num_times, 12)

    for i in range(num_times):
        states = []
        for j in range(num_servers):
            states.extend([INLET_TEMPERATURE[i, j], SERVER_TEMPERATURE[i, j], OUTLET_TEMPERATURE[i, j]])
        X[i, :] = np.array(states)

    for i in range(num_times):
        inputs = [Trcu[i], Qrcu[i]]
        for j in range(num_servers):
            inputs.append(SERVER_POWER[i, j])  # Use append, not extend
        U[i, :] = np.array(inputs)

    # compute predicted output and error for each time step and server
    for i in range(num_times - 1):
        Y_hat = Ac@X[i, :].T + Bc@U[i, :].T  # predicted output
        Y = X[i + 1, :].T  # actual output at next time step
        E = Y - Y_hat

    euclidean_norm = np.linalg.norm(E)

    fun = lambda th: PEM(th, X, U, Ac, Bc, N=N, maxiter=maxiter)  # variable parameter = th 
    res = scipy.optimize.minimize(fun, initial_params, method="L-BFGS-B", bounds=bounds,
                   options={"maxiter": maxiter})

    # Calculate the mean squared error between predicted and actual values
    mse = euclidean_norm / len(Y)

    return mse,E 

# Least squares optimization
def fit_pem(X, U, N, initial_params, bounds, alpha, maxiter):

    fun = lambda th: PEM(th, X, U, N=N, alpha=alpha)  # variable parameter = th 
    res = scipy.optimize.minimize(fun, initial_params, method="L-BFGS-B", bounds=bounds,
                   options={"maxiter": maxiter})
    return res


# steps to implement PEM
# Step 1: Initialise the parameters (done above)
# step 2: Estimate the time-varying coefficients using PEM
#---- tbc ----

# Step 3: Check stability of the model
# Calculate eigenvalues and eigenvectors of Ad
eigenvalues_Ac, eigenvectors_Ac = np.linalg.eig(Ac)
#print("eigenvalues of Ac:", eigenvalues_Ac)

eigenvalues_A, eigenvectors_A = np.linalg.eig(A)
#print("eigenvalues of Ad:", eigenvalues_A)

if np.all(np.abs(eigenvalues_Ac) < 0):
    print("The continuous-time system is stable.")
else:
    print("The continuous-time system is unstable.")


if np.all(np.abs(eigenvalues_A) < 1):
    print("The discrete-time system is stable.")
else:
    print("The discrete-time system is unstable.")  

#print("A matrix:", A)
print("A matrix shape:", np.shape(A))
print("A matrix ranks:", np.linalg.matrix_rank(A))

# Step 3: Validate the model


# Section 2: model the rack using black-box model input-output data
# Call for PEM using SIPPY for SS identification (THIS IS FOR BLACK BOX IDENTIFICATION NOT GREY BOX)
# input = [Trcu, Qrcu, SERVER_POWER.T]  # input vector [Qrcu; Trcu; Ps]
# output = Xmeas

# # METHOD = ["N4SID", "CVA", "MOESP", "PARSIM-S", "PARSIM-P", "PARSIM-K"]
# # METHOD = ["N4SID", "CVA", "MOESP"]
# METHOD = ["N4SID"]
# lege = ["System"]
# for i in range(len(METHOD)):
#     method = METHOD[i]
#     sys_id = system_identification(output, input, method, SS_fixed_order=30)
#     xid, yid = fsetSIM.SS_lsim_process_form(
#         sys_id.A, sys_id.B, sys_id.C, sys_id.D, input, sys_id.x0
#     )
#     #
#     plt.plot(len(Xmeas), yid[0])
#     plt.show(block=False)
#     lege.append(method)
# plt.legend(lege)



In [ ]:
# Simulate the output temperature 
Trcu = Trcu.reshape(-1, 1)  # Reshape to (180, 1)
Qrcu = Qrcu.reshape(-1, 1)  # Reshape to (180, 1)
SERVER_POWER = SERVER_POWER.reshape(-1, 10)  # Reshape to (180, 10)

input = np.concatenate([Trcu, Qrcu, SERVER_POWER], axis=1)  # input vector [Qrcu; Trcu; Ps] = [(180,1), (180,1), (180,10)]
print("Input shapes:", [i.shape for i in input])
print("input length:", len(input))

x = np.zeros((len(input), 1))  # Initialize state vector
y = np.zeros((len(input), 1))  # Initialize output vector
u = np.zeros((len(input), 1))  # Initialize input vector

# Assign inputs to u vector
for i in range(len(input)):
    u[i] = input[i]

sim_time = np.shape(inputs,1) # simulate the output for 60 steps

# simulate the system
for i in range(sim_time):
    x[i+1] = A@x[i] + B@u[i]
    #y[i+1] = C@x[i+1] + D@u[i]

# plot the output temperatures over time
plt.plot(x)
plt.xlabel('Time')
plt.ylabel('Output Temperature')
